GEO

In [1]:
from shapely.geometry import Point
from os import path
from json import load
from geopy.distance import geodesic
from PIL import Image
import geopandas as gpd, pickle, time, matplotlib.pyplot as plt, io, gc, logging

In [2]:
plt.switch_backend('agg')

regioni_path = "C:\\Users\\AleCe\\Documents\\Progetti\\TrasparentDisplay\\APP\\files\\shapefiles.json"
regioni_gdfs = "C:\\Users\\AleCe\\Documents\\Progetti\\TrasparentDisplay\\APP\\files\\gdfs.geojson.pkl"
image_path = "C:\\Users\\AleCe\\Documents\\Progetti\\TrasparentDisplay\\APP\\files\\image.bmp"
regioni_list = []
minimum_distance = 50

# regioni_pkl, regioni_json
with open(regioni_path, 'r') as f:
    obj = load(f)

    global regioni_json
    regioni_json = gpd.GeoDataFrame.from_features(obj)
    regioni_list = list(set(regioni_json["nome_reg"].values))

    if not path.exists(regioni_gdfs):
        print("File not found")
    else:
        with open(regioni_gdfs, 'rb') as f:
            time1 = time.time()
            global regioni_pkl
            regioni_pkl = pickle.load(f)
            time2 = time.time()

            print("Time to load pickle file: ", (time2 - time1))

Time to load pickle file:  6.473251819610596


In [3]:
# ALL FUNCTIONS

def get_regione(lon : float, lat : float) -> str:
    """
    Funziona che ritorna la regione corrente
    :param lon: Longitudine.
    :param lat: Latitudine.
    :return: La regione in cui il punto fa parte
    """
    point = Point(lon, lat)
    filtered_gdf = regioni_json[regioni_json.contains(point)]
    return list(dict(filtered_gdf.nome_reg).values())[0]

def find_nearby_roads(gdf : object, coordinate : tuple[float, float], buffer_distance : float = 0.001) -> object:
    """
    Funzione che ha lo scopo di ritorna tutte le strade vicine al punto
    :param gdf: GeoDataFrame, recuperato precedentemente.
    :param coordinate: Coordinate Longitudine, Latitudine.
    :return: GeoDataFrame della mappa (più precisa)
    """
    buffer_distance = (buffer_distance / 100000) if buffer_distance > 0 else buffer_distance

    point = Point(coordinate)

    # Applica un buffer al punto per ottenere una zona circostante
    return gdf[gdf.intersects(point.buffer(buffer_distance))]

def find_nearest_road(gdf : object, coordinate : tuple[float, float]) -> tuple[float, float]:
    """
    Funzione che ha lo scopo di ritornare la strada più vicina al punto
    :param gdf: GeoDataFrame, recuperato precedentemente.
    :param coordinate: Coordinate Longitudine, Latitudine.
    :return: Coordinate della strada più vicina
    """
    nearest_distance = float('inf') # Valore infinito
    nearest_geometry = None
    point = Point(coordinate)

    # Itera sul GeoDataFrame per trovare la geometria più vicina al punto
    for idx, row in gdf.iterrows():
        distance = row.geometry.distance(point)
        if distance < nearest_distance:
            nearest_distance = distance
            nearest_geometry = row.geometry
    
     # Crea un punto sulla strada più vicina alla posizione del punto di interesse
    new_point = nearest_geometry.interpolate(nearest_geometry.project(Point(coordinate)))

    return (float(new_point.x), float(new_point.y))

def get_minDistance(coordinate_lat : float, coordinate_lon : float, bbox: tuple[float, float, float, float]) -> (tuple[float, float, float, float], list): # type: ignore
        """
        Funzione che calcola la distanza minima dal punto ai bordi della bbox
        :param coordinate_lat: Latitudine del punto.
        :param coordinate_lon: Longitudine del punto.
        :param bbox: BoundyBox dell'aerea.
        :return: Distanza minima, Punto più vicino, Buffers
        """
        y_min, x_min, y_max, x_max = bbox

        # Calcola le coordinate dei vertici della bbox & coordinata
        bbox_vertices = [
            (coordinate_lon, y_min),  # Vertice in basso a sinistra
            (x_max, coordinate_lat),  # Vertice in basso a destra
            (coordinate_lon, y_max),  # Vertice in alto a destra
            (x_min, coordinate_lat)   # Vertice in alto a sinistra
        ]

        buffers = []
        # 0 => DOWN
        # 1 => RIGHT
        # 2 => UP
        # 3 => LEFT

        # Calcola la distanza minima dal punto ai bordi della bbox
        min_distance = float('inf')  # Inizializza con un valore elevato
        for vertex_lon, vertex_lat in bbox_vertices:

            buffers.append((coordinate_lat - vertex_lat) if(coordinate_lat - vertex_lat)!=0 else (coordinate_lon - vertex_lon))

            distance = geodesic((coordinate_lat, coordinate_lon), (vertex_lat, vertex_lon)).meters
            if distance < min_distance:
                min_distance = distance
                nearest_point = (vertex_lon, vertex_lat)
        
        # self._logger.info(msg = f"Distanza minima: {min_distance} meters")
        # self._logger.info(msg = f"Buffers (DOWN, RIGHT, UP, LEFT): {buffers}")

        return min_distance, nearest_point, buffers

def roads_to_image(roads_gdf : object, coordinate : tuple[float, float], margin : float, bbox : tuple[float, float, float, float], dpi: int) -> tuple[object, list]:
    """
    Funzione che ha lo scopo di ritornare l'immagine della mappa
    :param roads_gdf: GeoDataFrame, recuperato precedentemente.
    :param coordinate: Coordinate Longitudine, Latitudine.
    :param zoom: Zoom dell'area.
    :param bbox: BoundyBox dell'aerea.
    :return: Immagine della mappa, BBox della mappa
    """
    coordinate_lon, coordinate_lat = coordinate

    target_width = 128
    target_height = 64
    
    y_min, x_min, y_max, x_max = [0, 0, 0, 0]

    # Crea un'immagine binaca con strade nere
    # cdef object fig
    # cdef object ax
    fig, ax = plt.subplots(figsize=(12, 6), dpi=dpi)
    ax.set_aspect('equal')

    # Rimuovi assi
    ax.axis('off')

    y_min, x_min, y_max, x_max = bbox

    if not(x_min <= coordinate_lon <= x_max and y_min <= coordinate_lat <= y_max):
        # Nuovo bbox
        bbox = y_min, x_min, y_max, x_max = coordinate_lat - margin/2, coordinate_lon - margin, coordinate_lat + margin/2, coordinate_lon + margin
    
    min_distance, nearest_point, buffers = get_minDistance(coordinate_lat, coordinate_lon, bbox)

    if min_distance < minimum_distance:
        bbox = y_min, x_min, y_max, x_max = coordinate_lat - (-buffers[2]), coordinate_lon - (-buffers[1]), coordinate_lat + (buffers[0]), coordinate_lon + (buffers[3])

        meters_per_degree_latitude = 111139
        # Calculate the distance in degrees
        distance_degrees = minimum_distance / meters_per_degree_latitude

        if coordinate_lat - y_min < distance_degrees:
            diff = distance_degrees - (coordinate_lat - y_min)
            y_min -= diff
            y_max -= diff
        elif y_max - coordinate_lat < distance_degrees:
            diff = distance_degrees - (y_max - coordinate_lat)
            y_min += diff
            y_max += diff
        
        if coordinate_lon - x_min < distance_degrees:
            diff = distance_degrees - (coordinate_lon - x_min)
            x_min -= diff
            x_max -= diff
        elif x_max - coordinate_lon < distance_degrees:
            diff = distance_degrees - (x_max + coordinate_lon)
            x_min += diff
            x_max += diff
        
        bbox = y_min, x_min, y_max, x_max
        
        _, nearest_point, _ = get_minDistance(coordinate_lat, coordinate_lon, bbox)

    # Filter the GeoDataFrame of roads for the desired bounding box and plot directly
    roads_gdf.cx[x_min:y_min, x_max:y_max].plot(ax=ax, color='black', linewidth=5)

    # Disegna il punto (la tua coordinata)
    ax.plot(coordinate_lon, coordinate_lat, marker='o', color='black', markersize=35)

    # Imposta lo sfondo bianco
    ax.set_facecolor('white')

    # Calcola i limiti degli assi
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)

    # Salva l'immagine
    buffer_img = io.BytesIO()
    plt.savefig(buffer_img, format='png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    del fig, ax
    gc.collect()

    # Rewind the buffer_img to the beginning
    buffer_img.seek(0)

    # Apri l'immagine dal buffer_img e ridimensiona
    with Image.open(buffer_img) as img:
        resized_image = img.resize((target_width, target_height))

    # Converti l'immagine ridimensionata in bytes
    with io.BytesIO() as output_buffer:
        resized_image.save(output_buffer, format='png')
        output_buffer.seek(0)
        image_bytes = output_buffer.getvalue()
    
    del img
    gc.collect()

    # Chiudi il buffer_img originale
    buffer_img.close()

    return image_bytes, list(bbox)

def convert_png_to_bmp(image_bytes : object) -> object:
    """
    Funzione che converte un'immagine PNG in BMP
    :param image_bytes: Immagine in bytes.
    :return: Immagine convertita in BMP.
    """
    try:
        # Open the PNG image file
        with Image.open(io.BytesIO(image_bytes)) as img:

            # Convert the image to RGB mode (if it's in indexed or RGBA mode)
            img = img.convert("RGB")

            # Save the image as BMP format
            bmp_buffer = io.BytesIO()
            img.save(bmp_buffer, format='BMP')
            bmp_buffer.seek(0)

            # Read the content of the buffer and return as bytes
            # bmp_bytes = bmp_buffer.getvalue()
            
            return bmp_buffer
    except Exception as e:
        print(f"Conversion failed: {e}")

def create_image(coordinate: tuple[float, float], bbox : list = [0,0,0,0], zoom : float = 0.004, buffer_distance: float = 500, dpi: int = 300) -> tuple[float, float]:
    """
    Funzione che genererà l'immagine/mappa del luogo adiacente alla posizione
    :param coordinate: Coordinate Longitudine, Latitudine.
    :param bbox: BoundyBox dell'aerea.
    :param zoom: Zoom dell'area.
    :param buffer_distance: Distanza dal punto per il recupero delle mappe.
    :return: Immagine, Coordinate del Punto (calcolato), BBox della mappa, Regione in cui si trova il Punto
    """
    lon, lat = coordinate
    regione = get_regione(lon, lat)

    nearby_roads = find_nearby_roads(gdf = regioni_pkl[regione], coordinate = coordinate, buffer_distance = buffer_distance)

    if nearby_roads.empty:
        raise Exception("No roads found near the point")
    
    coordinate : tuple[float, float] = find_nearest_road(gdf = nearby_roads, coordinate = coordinate)

    image, bbox = roads_to_image(roads_gdf = nearby_roads, coordinate = coordinate, margin = zoom, bbox = tuple(bbox), dpi = dpi)

    image = convert_png_to_bmp(image)

    with open(image_path, "wb") as f:
        f.write(image.getvalue())

    return coordinate, list(bbox)

In [15]:
from PIL import Image, ImageSequence
import os

GIF_PATH = "mappa.gif"

from PIL import Image
import io

def convert_png_to_bmp_v2(image_bytes: bytes) -> Image.Image:
    """
    Converte un'immagine PNG (in bytes) in BMP e restituisce un oggetto PIL.Image.
    :param image_bytes: Immagine PNG in bytes
    :return: Oggetto PIL.Image in formato BMP
    """
    try:
        # Apri l'immagine dai bytes
        with Image.open(io.BytesIO(image_bytes)) as img:
            # Converti sempre in RGB (alcuni formati hanno palette o alpha)
            img = img.convert("RGB")

            # Salva come BMP in memoria
            bmp_buffer = io.BytesIO()
            img.save(bmp_buffer, format="BMP")
            bmp_buffer.seek(0)

            # Riapri come PIL.Image
            return Image.open(bmp_buffer)

    except Exception as e:
        print(f"Conversion failed: {e}")
        return None

def append_to_gif(new_image: Image.Image, gif_path: str = GIF_PATH, max_frames: int = 15, duration: int = 400):
    """
    Aggiunge un frame a una GIF esistente, o la crea se non esiste.
    :param new_image: Oggetto PIL.Image da aggiungere come frame
    :param gif_path: Percorso della gif
    :param max_frames: Numero massimo di frame nella gif
    :param duration: Durata di ogni frame in ms
    """
    if os.path.exists(gif_path):
        # Carica la gif esistente
        with Image.open(gif_path) as im:
            frames = [frame.copy() for frame in ImageSequence.Iterator(im)]
        
        # Aggiungi nuovo frame
        frames.append(new_image.convert("RGB"))
        
        # Taglia a max_frames
        if len(frames) > max_frames:
            frames = frames[-max_frames:]
    else:
        # Nuova GIF
        frames = [new_image.convert("RGB")]

    # Salva la gif aggiornata
    frames[0].save(
        gif_path,
        save_all=True,
        append_images=frames[1:],
        duration=duration,
        loop=0
    )

def create_image_v2(coordinate: tuple, bbox : list = [0,0,0,0], zoom : float = 0.004, buffer_distance: float = 500, dpi: int = 300) -> tuple:
    """
    Funzione che genererà l'immagine/mappa del luogo adiacente alla posizione
    e la concatenerà in una GIF (max 15 frame).
    """
    lon, lat = coordinate
    regione = get_regione(lon, lat)

    nearby_roads = find_nearby_roads(gdf = regioni_pkl[regione], coordinate = coordinate, buffer_distance = buffer_distance)

    if nearby_roads.empty:
        raise Exception("No roads found near the point")
    
    coordinate = find_nearest_road(gdf = nearby_roads, coordinate = coordinate)

    image, bbox = roads_to_image(
        roads_gdf = nearby_roads, 
        coordinate = coordinate, 
        margin = zoom, 
        bbox = tuple(bbox), 
        dpi = dpi
    )

    # Converti in BMP/PIL
    pil_img = convert_png_to_bmp_v2(image)

    # Salva l'immagine in una cartella temporanea
    temp_image_path = f"tmp/{coordinate[0]}_{coordinate[1]}.bmp"
    pil_img.save(temp_image_path)

    # Aggiungi alla gif
    append_to_gif(pil_img)

    return coordinate, list(bbox)


MAIN

In [17]:
import time

coordinates = [[44.979213842916465, 8.566036475980964],
[44.979299995924585, 8.566131204721652],
[44.97960312626724, 8.566455991134188],
[44.979887111683865, 8.566428926334977],
[44.980218959668356, 8.565973318027964],
[44.98068800591733, 8.565580867755388],
[44.98102941744561, 8.565301191847253],
[44.98135806962276, 8.56508917299625],
[44.98158460408342, 8.56467416966037],
[44.981855816779316, 8.564299753622562],
[44.98148568662383, 8.564177964377313],
[44.98101026156943, 8.564078733548387],
[44.98072308894788, 8.563411119401687],
[44.98051248967389, 8.562901391646172],
[44.98023171140059, 8.562175124169936],
[44.97993177003429, 8.561430825951726],
[44.979717990393254, 8.560862451367381],
[44.979497811876534, 8.559915165021108],
[44.97943400040123, 8.55910770427052],
[44.97920426094329, 8.558137858271923],
[44.97857566342873, 8.557001103972572],
[44.9781672244454, 8.556216208311506],
[44.97758009901368, 8.555598206892364],
[44.97683979351433, 8.555061414024719],
[44.976511120282055, 8.554935101206594],
[44.97588886973584, 8.554948633825195],
[44.97531128852977, 8.554957652985971],
[44.97451032234257, 8.554899014419266],
[44.97388166833287, 8.554452431775037],
[44.97296260787555, 8.5533066583467],
[44.972467964985206, 8.552661605705975]]

bbox = [0,0,0,0]

for idx, coordinate in enumerate(coordinates):
    # print(idx)

    latitude, longitude = coordinate[0], coordinate[1]

    print(f"Lat: {latitude}, Long: {longitude}")
    _, bbox = create_image_v2(coordinate = (longitude, latitude), bbox = bbox, dpi=200, zoom=0.003)

    time.sleep(0.5)

Lat: 44.979213842916465, Long: 8.566036475980964
Lat: 44.979299995924585, Long: 8.566131204721652
Lat: 44.97960312626724, Long: 8.566455991134188
Lat: 44.979887111683865, Long: 8.566428926334977
Lat: 44.980218959668356, Long: 8.565973318027964
Lat: 44.98068800591733, Long: 8.565580867755388
Lat: 44.98102941744561, Long: 8.565301191847253
Lat: 44.98135806962276, Long: 8.56508917299625
Lat: 44.98158460408342, Long: 8.56467416966037
Lat: 44.981855816779316, Long: 8.564299753622562
Lat: 44.98148568662383, Long: 8.564177964377313
Lat: 44.98101026156943, Long: 8.564078733548387
Lat: 44.98072308894788, Long: 8.563411119401687
Lat: 44.98051248967389, Long: 8.562901391646172
Lat: 44.98023171140059, Long: 8.562175124169936
Lat: 44.97993177003429, Long: 8.561430825951726
Lat: 44.979717990393254, Long: 8.560862451367381
Lat: 44.979497811876534, Long: 8.559915165021108
Lat: 44.97943400040123, Long: 8.55910770427052
Lat: 44.97920426094329, Long: 8.558137858271923
Lat: 44.97857566342873, Long: 8.5570

In [10]:
# get number of gif frame

from PIL import Image, ImageSequence

def get_gif_frame_count(gif_path):
    with Image.open(gif_path) as img:
        return sum(1 for _ in ImageSequence.Iterator(img))

gif_path = "C:\\Users\\AleCe\\Documents\\Progetti\\TrasparentDisplay\\APP\\mappa.gif"
frame_count = get_gif_frame_count(gif_path)
print(f"Number of frames in GIF: {frame_count}")


Number of frames in GIF: 15
